## Finetune tiny-llama in Google Colab with unsloth - dolly-15k dataset

Below notebook is mostly replicates an example given in the [official unsloth example ](https://github.com/unslothai/unsloth)

I have only made few minor changes

In [1]:
import torch
from datasets import load_dataset
from unsloth import FastLanguageModel

In [ ]:
max_seq_length = 4096 # Choose any as, unsloth supports RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/tinyllama-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Supported models - 

https://github.com/unslothai/unsloth?tab=readme-ov-file#documentation

```py
# 4bit pre quantized models we support - 4x faster downloading!
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
]

```

TinyLlama's internal maximum sequence length is 2048. 

However, RoPE Scaling is used here to extend it to 4096 with Unsloth

Unsloth implements RoPE with Triton to accelerate models further.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

**[NOTE]** We set `gradient_checkpointing=False` ONLY for TinyLlama since Unsloth saves tonnes of memory usage. This does NOT work for `llama-2-7b` or `mistral-7b` since the memory usage will still exceed Tesla T4's 15GB. GC recomputes the forward pass during the backward pass, saving loads of memory.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    use_gradient_checkpointing = False, # With Unsloth, we can turn this off!
    random_state = 3407,
    max_seq_length = max_seq_length,
)

Here I use databricks-dolly-15k which is an open source dataset of instruction-following records generated by thousands of Databricks employees in several of the behavioral categories outlined in the InstructGPT paper, including brainstorming, classification, closed QA, generation, information extraction, open QA, and summarization.

In [ ]:
dolly_15k_prompt = """Below is an instruction that describes a task, paired with a context for that instruction. Write a response that appropriately completes the instruction.

### Instruction:
{}

### Context:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    context = examples.get("context", [None] * len(instructions))
    response = examples["response"]
    texts = []
    for instruction, context_value, response_value in zip(instructions, context, response):
        if context_value:  # Use the context if it exists
            text = dolly_15k_prompt.format(instruction, context_value, response_value)
        else:
            text = ""  # Or use an empty string (or another placeholder) if context is missing
        texts.append(text)
    return {"text": texts}


pass

dataset = load_dataset("databricks/databricks-dolly-15k", split = "train")

dataset = dataset.map(formatting_prompts_func, batched = True,)

# Filter out rows where 'text' is empty
dataset = dataset.filter(lambda example: example['text'] != "")

<a name="Train"></a>

### Train the model

Now use TRL’s SFTTrainer to fine-tune models. 

[TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). 

We also support TRL's `DPOTrainer`! See our DPO tutorial on a free Google Colab instance [here](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing).

------
Official Doc of SFTTrainer - https://huggingface.co/docs/trl/main/en/sft_trainer

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers.utils import logging
logging.set_verbosity_info()

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = True, # Packs short sequences together to save time!
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 1,
        learning_rate = 2e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
inputs = tokenizer(
[
    dolly_15k_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
]*1, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

In [ ]:
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model") # Online saving

To save to `GGUF` / `llama.cpp`, or for model merging, use `model.merge_and_unload` first, then save the model. Maxime Labonne's [llm-course](https://mlabonne.github.io/blog/posts/Quantize_Llama_2_models_using_ggml.html) has a nice tutorial on converting HF to GGUF! This [issue](https://github.com/ggerganov/llama.cpp/issues/3097) might be helpful for more info.

In [ ]:
model = model.merge_and_unload()

Now if you want to load the LoRA adapters we just saved, we can!

In [ ]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "lora_model")

Finally, we can now do some inference on the loaded model.

In [ ]:
inputs = tokenizer(
[
    dolly_15k_prompt.format(
        "What is the famous tower in France called?", # instruction
        "", # input
        "", # output
    )
]*1, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)